In [ ]:
from tkinter import *
from tkinter.ttk import Combobox
import time
from tkinter import messagebox
import sqlite3
import re 

try:
    conobj = sqlite3.connect(database = 'bank.sqlite')
    curobj = conobj.cursor()
    curobj.execute('create table acn(acn_no integer primary key autoincrement, acn_name text, acn_pass text, acn_email text, acn_mob text, acn_bal float, acn_opendate text, acn_gender text)')
    conobj.close()
    print("table created")
except:
    print("something went wrong, may be the table already exists")

win = Tk()
win.attributes('-fullscreen', True)
win.configure(bg = '#db6723')

win.resizable(width = False, height = False)

title = Label(win,text = 'Banking Automation', font=('Aerial',42, 'bold','underline'), bg = '#db6723', fg = 'black')
title.pack()

date = Label(win,text = time.strftime('%d %B %Y, %A'), font=('Aerial',22, 'bold'), bg = '#db6723', fg = 'black')
date.place(relx = .8,rely=.001)

def main_screen():
    frm = Frame(win)
    frm.configure(bg = '#33b8ad')
    frm.place(relx = 0, rely = .1, relwidth = 1, relheight = .85)

    def forgotpass():
        frm.destroy()
        forgotpass_screen()

    def newuser():
        frm.destroy()
        newuser_screen()

    def login():
        global gacn
        gacn = e_entry.get()
        pwd = e_pass.get()
        if len(gacn) == 0 or len(pwd) == 0:
            messagebox.showwarning('Validation','Empty entries are not allowed \n please fill the entries')
        else:
            conobj = sqlite3.connect(database = 'bank.sqlite')
            curobj = conobj.cursor()
            curobj.execute('select * from acn where acn_no =? and acn_pass=?',(gacn, pwd))
            tup = curobj.fetchone()
            conobj.commit()
            conobj.close()
            if tup == None:
                messagebox.showerror('Login', 'Invalid ACN/PASS')

            else:
                frm.destroy()
                welcome_screen()
                

    def clearclick():
        e_entry.delete(0,'end')
        e_pass.delete(0,'end')
        e_entry.focus()

    lbl_acn = Label(frm, text = 'ACN', font = ('arial', 20, 'bold'), bg= '#33b8ad', fg=  'black')
    lbl_acn.place(relx= .1, rely= .1)

    e_entry = Entry(frm, font = ('arial', 20, 'bold'), bd = 2)
    e_entry.place(relx= .2, rely= .1)
    e_entry.focus()

    lbl_pass = Label(frm, text = 'Password', font = ('arial', 20, 'bold'), bg= '#33b8ad', fg=  'black')
    lbl_pass.place(relx= .1, rely= .17)

    e_pass = Entry(frm, font = ('arial', 20, 'bold'), bd = 2, show = '*')
    e_pass.place(relx= .2, rely= .17)

    btn_login = Button(frm, text='Login',font = ('arial', 20, 'bold'), command = login )
    btn_login.place(relx= .2, rely = .25)

    btn_clear = Button(frm, text='Clear',font = ('arial', 20, 'bold'), command = clearclick )
    btn_clear.place(relx= .315, rely = .25)

    btn_fgt_pswd = Button(frm, text='Forgot Password',font = ('arial', 20, 'bold'), command=forgotpass )
    btn_fgt_pswd.place(relx= .22, rely = .32)

    btn_new = Button(frm, text='Open New Account',font = ('arial', 20, 'bold'), command = newuser )
    btn_new.place(relx= .2125, rely = .39)


def forgotpass_screen():
    frm = Frame(win)
    frm.configure(bg = '#33b8ad')
    frm.place(relx = 0, rely = .1, relwidth = 1, relheight = .85)

    def back():
        frm.destroy()
        main_screen()

    def forgotpass_db():
        account_no = e_acn.get()
        email = e_email.get()
        mob = e_mob.get()
        newpass = e_newpass.get()

        conobj = sqlite3.connect(database = 'bank.sqlite')
        curobj = conobj.cursor()
        curobj.execute('update acn set acn_pass =? where acn_no =? and acn_email=? and acn_mob=?',(newpass, account_no, email, mob))
        conobj.commit()
        conobj.close()

        conobj = sqlite3.connect(database = 'bank.sqlite')
        curobj = conobj.cursor()
        curobj.execute('select acn_pass from acn where acn_no =? and acn_email=? and acn_mob=?',(account_no, email, mob))
        tup = curobj.fetchone()
        conobj.commit()
        conobj.close()

        if tup==None:
            messagebox.showerror('Forgot Pass', 'Record not found')

        else:
            messagebox.showinfo('Forgot Pass', f'Your password has been changed successfully \n Your new password is {tup[0]}')
            main_screen()

    btn_back = Button(frm, text='back',font = ('arial', 20, 'bold'), command=back )
    btn_back.place(relx= 0, rely = 0)

    lbl_acn = Label(frm, text = 'ACN', font = ('arial', 20, 'bold'), bg= '#33b8ad', fg=  'black')
    lbl_acn.place(relx= .1, rely= .1)

    e_acn = Entry(frm, font = ('arial', 20, 'bold'), bd = 2)
    e_acn.place(relx= .22, rely= .1)
    e_acn.focus()

    lbl_email = Label(frm, text = 'Email', font = ('arial', 20, 'bold'), bg= '#33b8ad', fg=  'black')
    lbl_email.place(relx= .1, rely= .17)

    e_email = Entry(frm, font = ('arial', 20, 'bold'), bd = 2)
    e_email.place(relx= .22, rely= .17)

    lbl_mob = Label(frm, text = 'Mob', font = ('arial', 20, 'bold'), bg= '#33b8ad', fg=  'black')
    lbl_mob.place(relx= .1, rely= .24)

    e_mob = Entry(frm, font = ('arial', 20, 'bold'), bd = 2)
    e_mob.place(relx= .22, rely= .24)

    lbl_newpass = Label(frm, text = 'New Password', font = ('arial', 20, 'bold'), bg= '#33b8ad', fg=  'black')
    lbl_newpass.place(relx= .1, rely= .31)

    e_newpass = Entry(frm, font = ('arial', 20, 'bold'), bd = 2)
    e_newpass.place(relx= .22, rely= .31)

    btn_sub = Button(frm, text='Submit',font = ('arial', 20, 'bold'), command = forgotpass_db )
    btn_sub.place(relx= .27, rely = .4)

def newuser_screen():
    frm = Frame(win)
    frm.configure(bg = '#33b8ad')
    frm.place(relx = 0, rely = .1, relwidth = 1, relheight = .85)

    def back():
        frm.destroy()
        main_screen()

    def newuser_db():
        name = e_name.get()
        email = e_email.get()
        mobile = e_mob.get()
        password = e_pass.get()
        gender = cb_gender.get()
        bal = 0
        opendate = time.strftime('%d %B %Y, %A')

        match = re.fullmatch("[6-9][0-9]{9}", mobile)

        if match==None:
            messagebox.showwarning("Validation", 'Invalid Mobile Number')
            return 

        match_email = re.fullmatch("[a-zA-Z0-9_]+@[a-zA-Z]+\.[a-zA-Z]", email)

        if match_email==None:
            messagebox.showwarning("Validation", 'Invalid Format of Email')
            return 
            
        conobj = sqlite3.connect(database = 'bank.sqlite')
        curobj = conobj.cursor()
        curobj.execute('''Insert into acn(acn_name, acn_pass, acn_email, acn_mob, acn_bal, acn_opendate, acn_gender)
                        values(?,?,?,?,?,?,?)''', (name, password, email, mobile, bal, opendate, gender))
        conobj.commit()
        conobj.close()

        conobj = sqlite3.connect(database = 'bank.sqlite')
        curobj = conobj.cursor()
        curobj.execute('''select max(acn_no) from acn''')
        tup = curobj.fetchone()
        conobj.commit()
        conobj.close()

        messagebox.showinfo('New User', f'Account Created Successfully... \n Your Account number is {tup[0]}')
        e_name
        e_email
        e_mob
        e_pass
        cb_gender
        
        
    btn_back = Button(frm, text='back',font = ('arial', 20, 'bold'), command=back )
    btn_back.place(relx= 0, rely = 0)

    lbl_name = Label(frm, text = 'Name', font = ('arial', 20, 'bold'), bg= '#33b8ad', fg=  'black')
    lbl_name.place(relx= .1, rely= .1)

    e_name = Entry(frm, font = ('arial', 20, 'bold'), bd = 2)
    e_name.place(relx= .22, rely= .1)
    e_name.focus()

    lbl_email = Label(frm, text = 'Email', font = ('arial', 20, 'bold'), bg= '#33b8ad', fg=  'black')
    lbl_email.place(relx= .1, rely= .17)

    e_email = Entry(frm, font = ('arial', 20, 'bold'), bd = 2)
    e_email.place(relx= .22, rely= .17)

    lbl_mob = Label(frm, text = 'Mob', font = ('arial', 20, 'bold'), bg= '#33b8ad', fg=  'black')
    lbl_mob.place(relx= .1, rely= .24)

    e_mob = Entry(frm, font = ('arial', 20, 'bold'), bd = 2)
    e_mob.place(relx= .22, rely= .24)

    lbl_pass = Label(frm, text = 'Password', font = ('arial', 20, 'bold'), bg= '#33b8ad', fg=  'black')
    lbl_pass.place(relx= .1, rely= .31)

    e_pass = Entry(frm, font = ('arial', 20, 'bold'), bd = 2, show = '*')
    e_pass.place(relx= .22, rely= .31)

    lbl_gender = Label(frm, text = 'Gender', font = ('arial', 20, 'bold'), bg= '#33b8ad', fg=  'black')
    lbl_gender.place(relx= .1, rely= .38)

    cb_gender = Combobox(frm,values = ['---select---', 'Male', 'Female', 'Trans', 'Others'], font = ('arial', 20, 'bold'))
    cb_gender.place(relx= .22, rely= .38)

    btn_sub = Button(frm, text='Submit',font = ('arial', 20, 'bold'), command = newuser_db)
    btn_sub.place(relx= .27, rely = .5)

def welcome_screen():
    frm = Frame(win)
    frm.configure(bg = '#33b8ad')
    frm.place(relx = 0, rely = .1, relwidth = 1, relheight = .85)

    def Logout():
        frm.destroy()
        main_screen()

    def details():
        ifrm = Frame(frm, highlightbackground = 'black', highlightthickness = 1)
        ifrm.configure(bg = 'white')
        ifrm.place(relx = .15, rely = .1, relwidth = .8, relheight = .85)

        lbl_wel_det = Label(ifrm, text = 'This is Details Screen', font = ('arial', 32, 'bold'), bg= 'white', fg=  'blue')
        lbl_wel_det.pack()

        conobj = sqlite3.connect(database = 'bank.sqlite')
        curobj = conobj.cursor()
        curobj.execute("select acn_email, acn_mob, acn_bal, acn_opendate from acn where acn_no =?", (gacn,))
        tup = curobj.fetchone()
        conobj.close()
    
        lbl_zero = Label(frm, text = f'Email: {tup[0]}', font = ('arial', 22, 'bold'), bg= 'white', fg=  'black')
        lbl_zero.place(relx= .2, rely= .3)
    
        lbl_one= Label(frm, text = f'Mobile No. : {tup[1]}', font = ('arial', 22, 'bold'), bg= 'white', fg=  'black')
        lbl_one.place(relx= .2, rely= .4)
    
        lbl_two = Label(frm, text = f'Current Balance : {tup[2]}', font = ('arial', 22, 'bold'), bg= 'white', fg=  'black')
        lbl_two.place(relx= .2, rely=.5)
    
        lbl_three = Label(frm, text = f'Open Date : {tup[3]}', font = ('arial', 22, 'bold'), bg= 'white', fg=  'black')
        lbl_three.place(relx= .2, rely= .6)

    def update():
        ifrm = Frame(frm, highlightbackground = 'black', highlightthickness = 1)
        ifrm.configure(bg = 'white')
        ifrm.place(relx = .15, rely = .1, relwidth = .8, relheight = .85)

        def save_changes_db():
            new_name = e_name.get()
            new_email = e_email.get()
            new_pass = e_pass.get()
            new_mob = e_mobile.get()

            match = re.fullmatch("[6-9][0-9]{9}", new_mob)
            if match==None:
                messagebox.showwarning("Validation", 'Enter a Valid Mobile Number')
                return 

            match_email = re.fullmatch("[a-zA-Z0-9_]+@[a-zA-Z]+\.[a-zA-Z]", new_email)
            if match_email==None:
                messagebox.showwarning("Validation", 'Invalid Format of Email')
                return 
            
            conobj = sqlite3.connect(database = 'bank.sqlite')
            curobj = conobj.cursor()
            curobj.execute("update acn set acn_name=?, acn_email=?, acn_pass=?, acn_mob=? where acn_no=? ", (new_name, new_email, new_pass, new_mob, gacn))
            conobj.commit()
            conobj.close()

            messagebox.showinfo('Update', "Changes have been saved successfully")
            welcome_screen()

        lbl_wel_det = Label(ifrm, text = 'This is Update Screen', font = ('arial', 22, 'bold'), bg= 'white', fg=  'blue')
        lbl_wel_det.pack()

        conobj = sqlite3.connect(database = 'bank.sqlite')
        curobj = conobj.cursor()
        curobj.execute("select acn_name, acn_email, acn_pass, acn_mob from acn where acn_no =?", (gacn,))
        tup = curobj.fetchone()
        conobj.close()

        lbl_name = Label(ifrm, text = 'Name', font = ('arial', 20, 'bold'), bg= 'powder blue', fg=  'black')
        lbl_name.place(relx= .1, rely= .2)
    
        e_name = Entry(ifrm, font = ('arial', 20, 'bold'), bd = 2)
        e_name.place(relx= .1, rely= .25)
        e_name.insert(0,tup[0])
    
        lbl_email = Label(ifrm, text = 'Email', font = ('arial', 20, 'bold'), bg= 'powder blue', fg=  'black')
        lbl_email.place(relx= .1, rely= .35)
    
        e_email = Entry(ifrm, font = ('arial', 20, 'bold'), bd = 2)
        e_email.place(relx= .1, rely= .4)
        e_email.insert(0,tup[1])
        
        lbl_pass = Label(ifrm, text = 'Password', font = ('arial', 20, 'bold'), bg= 'powder blue', fg=  'black')
        lbl_pass.place(relx= .5, rely= .2)
    
        e_pass = Entry(ifrm, font = ('arial', 20, 'bold'), bd = 2)
        e_pass.place(relx= .5, rely= .25)
        e_pass.insert(0,tup[2])
        
        lbl_mobile = Label(ifrm, text = 'Mobile', font = ('arial', 20, 'bold'), bg= 'powder blue', fg=  'black')
        lbl_mobile.place(relx= .5, rely= .35)

        e_mobile = Entry(ifrm, font = ('arial', 20, 'bold'), bd = 2)
        e_mobile.place(relx= .5, rely= .4)
        e_mobile.insert(0,tup[3])
        
        btn_save = Button(ifrm, text='Save Changes',font = ('arial', 20, 'bold'), command = save_changes_db )
        btn_save.place(relx= .8, rely = .8)

    def deposit():
        ifrm = Frame(frm, highlightbackground = 'black', highlightthickness = 1)
        ifrm.configure(bg = 'white')
        ifrm.place(relx = .15, rely = .1, relwidth = .8, relheight = .85)

        def deposit_db():
            amount = float(e_amount.get())

            conobj = sqlite3.connect(database = 'bank.sqlite')
            curobj = conobj.cursor()
            curobj.execute("update acn set acn_bal= acn_bal+? where acn_no=? ", (amount, gacn))
            conobj.commit()
            conobj.close()
            messagebox.showinfo('Deposit', f"{amount} deposited successfully")

        lbl_wel_det = Label(ifrm, text = 'This is Deposit Screen', font = ('arial', 22, 'bold'), bg= 'white', fg=  'blue')
        lbl_wel_det.pack()

        lbl_amount = Label(ifrm, text = 'Amount', font = ('arial', 20, 'bold'), bg= 'powder blue', fg=  'black')
        lbl_amount.place(relx= .3, rely= .2)

        e_amount = Entry(ifrm, font = ('arial', 20, 'bold'), bd = 2)
        e_amount.place(relx= .3, rely= .25)

        btn_sub = Button(ifrm, text='Submit',font = ('arial', 20, 'bold'), command = deposit_db)
        btn_sub.place(relx= .45, rely = .5)
        
    def withdraw():
        ifrm = Frame(frm, highlightbackground = 'black', highlightthickness = 1)
        ifrm.configure(bg = 'white')
        ifrm.place(relx = .15, rely = .1, relwidth = .8, relheight = .85)

        lbl_wel_det = Label(ifrm, text = 'This is Withdraw Screen', font = ('arial', 22, 'bold'), bg= 'white', fg=  'blue')
        lbl_wel_det.pack()

        def withdraw_db():
            amount = float(e_amount.get())

            conobj = sqlite3.connect(database = 'bank.sqlite')
            curobj = conobj.cursor()
            curobj.execute("select acn_bal from acn where acn_no =?", (gacn,))
            tup = curobj.fetchone()
            conobj.close()

            bal = tup[0]
            if bal>=amount:
                conobj = sqlite3.connect(database = 'bank.sqlite')
                curobj = conobj.cursor()
                curobj.execute("update acn set acn_bal= acn_bal-? where acn_no=? ", (amount, gacn))
                conobj.commit()
                conobj.close()
                messagebox.showinfo('Withdraw', f"{amount} withdrawn successfully")

            else:
                messagebox.showwarning('Withdraw', "Insufficient Balance")
            
        lbl_amount = Label(ifrm, text = 'Amount', font = ('arial', 20, 'bold'), bg= 'powder blue', fg=  'black')
        lbl_amount.place(relx= .3, rely= .2)

        e_amount = Entry(ifrm, font = ('arial', 20, 'bold'), bd = 2)
        e_amount.place(relx= .3, rely= .25)

        btn_sub = Button(ifrm, text='Submit',font = ('arial', 20, 'bold'), command = withdraw_db)
        btn_sub.place(relx= .45, rely = .5)

    def transfer():
        ifrm = Frame(frm, highlightbackground = 'black', highlightthickness = 1)
        ifrm.configure(bg = 'white')
        ifrm.place(relx = .15, rely = .1, relwidth = .8, relheight = .85)

        def transfer_db():
            acn = e_rec_acn.get()
            amount = int(e_amount.get())

            conobj = sqlite3.connect(database = 'bank.sqlite')
            curobj = conobj.cursor()
            curobj.execute("select acn_bal from acn where acn_no =?", (acn,))
            tup = curobj.fetchone()
            conobj.close()

            if acn==gacn:
                messagebox.showwarning('Transfer', "Sender and Reciver can't be same!")
                return
            
            if tup==None:
                messagebox.showwarning('Transfer', "Invalid Receiver ACN")

            else:
                
                conobj = sqlite3.connect(database = 'bank.sqlite')
                curobj = conobj.cursor()
                curobj.execute("select acn_bal from acn where acn_no =?", (gacn,))
                tup = curobj.fetchone()
                conobj.close()
    
                bal = tup[0]
                if bal>=amount:
                    conobj = sqlite3.connect(database = 'bank.sqlite')
                    curobj = conobj.cursor()
                    curobj.execute("update acn set acn_bal= acn_bal-? where acn_no=? ", (amount, gacn))
                    conobj.commit()
                    conobj.close()
    
                    conobj = sqlite3.connect(database = 'bank.sqlite')
                    curobj = conobj.cursor()
                    curobj.execute("update acn set acn_bal= acn_bal+? where acn_no=? ", (amount, acn))
                    conobj.commit()
                    conobj.close()
    
                    messagebox.showinfo('Transfer', f"{amount} Transferred successfully")
    
                else:
                    messagebox.showwarning('Withdraw', "Insufficient Balance")

                
        lbl_wel_det = Label(ifrm, text = 'This is Transfer Screen', font = ('arial', 22, 'bold'), bg= 'white', fg=  'blue')
        lbl_wel_det.pack()

        lbl_amount = Label(ifrm, text = 'Amount', font = ('arial', 20, 'bold'), bg= 'powder blue', fg=  'black')
        lbl_amount.place(relx= .2, rely= .2)

        e_amount = Entry(ifrm, font = ('arial', 20, 'bold'), bd = 2)
        e_amount.place(relx= .2, rely= .25)
        e_amount.focus()
        
        lbl_rec_acn = Label(ifrm, text = "Receiver's ACN", font = ('arial', 20, 'bold'), bg= 'powder blue', fg=  'black')
        lbl_rec_acn.place(relx= .5, rely= .2)

        e_rec_acn = Entry(ifrm, font = ('arial', 20, 'bold'), bd = 2)
        e_rec_acn.place(relx= .5, rely= .25)

        btn_sub = Button(ifrm, text='Transfer',font = ('arial', 20, 'bold'), command = transfer_db)
        btn_sub.place(relx= .8, rely = .8)

        # command = transfer_db
    

    conobj = sqlite3.connect(database = 'bank.sqlite')
    curobj = conobj.cursor()
    curobj.execute("select acn_name from acn where acn_no =?", (gacn,))
    tup = curobj.fetchone()
    conobj.close()

    name  = tup[0]
    lbl_wel = Label(frm, text = f'Welcome, {name}', font = ('arial', 38, 'bold'), bg= '#33b8ad', fg=  'black')
    lbl_wel.place(relx= 0, rely= 0)

    btn_logout = Button(frm, width = 10, text='Logout',font = ('arial', 20, 'bold'), command = Logout )
    btn_logout.place(relx= .9, rely = 0)

    btn_details = Button(frm, width = 10, text='Details',font = ('arial', 20, 'bold'), command = details)
    btn_details.place(relx= 0, rely = .1)
    
    btn_update = Button(frm, width = 10, text='Update',font = ('arial', 20, 'bold'), command = update)
    btn_update.place(relx= 0, rely = .2)

    btn_deposit = Button(frm, width = 10, text='Deposit',font = ('arial', 20, 'bold'), command = deposit)
    btn_deposit.place(relx= 0, rely = .3)

    btn_withdraw = Button(frm, width = 10, text='Withdraw',font = ('arial', 20, 'bold'), command = withdraw)
    btn_withdraw.place(relx= 0, rely = .4)
    
    btn_transfer = Button(frm, width = 10, text='Transfer',font = ('arial', 20, 'bold'), command = transfer)
    btn_transfer.place(relx= 0, rely = .5)
    

main_screen()
win.mainloop()


something went wrong, may be the table already exists


In [4]:
conobj = sqlite3.connect(database = 'bank.sqlite')
curobj = conobj.cursor()
curobj.execute('''select * from acn''')
tup = curobj.fetchall()
print(tup)
conobj.commit()
conobj.close()

[(1, 'vinayak', 'ok', 'kanojiavinayak024@gmail.com', '9454290720', 1000.0, '27 July 2024, Saturday', 'Male'), (2, 'rajesh', 'youknow', 'rajesh@gmail.com', '9988223344', 0.0, '27 July 2024, Saturday', 'Trans'), (3, 'vidyut', 'bye', 'hello@123', '5678944732', 0.0, '01 August 2024, Thursday', 'Others')]


In [12]:
import re
mobile = input("Enter your mobile number")


Something went wrong, maybe the table already exists
